# Proteomics Data Analysis: From Raw Data to Disease Prediction

## 🧬 Welcome to Bioinformatics Machine Learning!

### Learning Objectives
In this notebook, you will learn:
1. **Data Quality Issues** in proteomics: missing values, batch effects, outliers
2. **Data Cleaning Strategies**: filtering, imputation, outlier detection
3. **Normalization Methods**: when and why to use different approaches
4. **Batch Effect Correction**: identifying and removing technical artifacts
5. **Machine Learning**: building classifiers for disease prediction
6. **Feature Importance**: identifying which proteins matter most

### The Biological Context
You're analyzing protein expression data from patient samples to predict disease status. Proteomics data is notoriously messy:
- Proteins have vastly different abundance levels (6+ orders of magnitude!)
- Missing values are common (proteins below detection limit)
- Technical variation (batch effects) can obscure biological signal
- Proper preprocessing is CRITICAL for meaningful results

---

## 📋 How to Use This Notebook in Google Colab

### Option 1: Upload to Google Colab
1. Go to [Google Colab](https://colab.research.google.com/)
2. Click **File → Upload notebook**
3. Choose this `.ipynb` file from your computer
4. The notebook will open in Colab

### Option 2: Open from GitHub (if hosted there)
1. Go to [Google Colab](https://colab.research.google.com/)
2. Click **File → Open notebook → GitHub**
3. Enter the repository URL

### Running the Notebook
- **Run a cell**: Click the play button (▶️) or press `Shift+Enter`
- **Run all cells**: Click **Runtime → Run all**
- **Restart runtime**: Click **Runtime → Restart runtime** if you need to start fresh

### Saving Your Work
- Colab auto-saves to Google Drive
- Download: **File → Download → Download .ipynb**
- Your changes won't affect the original file unless you explicitly save over it

### Tips
- Look for **TODO** comments - these are exercises for you to complete!
- Read the markdown explanations before running code cells
- Try modifying parameters to see how results change
- If you get stuck, restart the runtime and run all cells from the top

---

## 1. Setup and Library Installation

First, let's import all the libraries we'll need. No special proteomics libraries required - we'll use standard scientific Python tools!

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("✓ All libraries imported successfully!")
print("✓ Random seed set to 42 for reproducibility")

## 2. Dataset Generation

### Understanding Proteomics Data

We're simulating a realistic proteomics experiment:
- **100 samples**: 50 healthy controls, 50 disease patients
- **500 proteins**: each with expression values
- **2 technical batches**: samples processed on different days
- **Realistic issues**: missing values, different protein abundances, batch effects, outliers

### Key Challenges We're Simulating:
1. **Missing values (15-20%)**: Some proteins aren't detected in all samples
2. **Dynamic range**: Proteins have different baseline expression levels
3. **Batch effects**: Technical variation between processing batches
4. **Outliers**: A few samples with unusual patterns (technical failures)
5. **Biological signal**: Disease-related changes in specific proteins

In [ ]:
def generate_proteomics_data():
    """
    Generate realistic simulated proteomics data with common data quality issues.
    
    Returns:
    --------
    df : DataFrame with protein expression values
    metadata : DataFrame with sample information (disease status, batch)
    """
    
    n_samples = 100
    n_proteins = 500
    
    # Create sample metadata
    sample_ids = [f'Sample_{i:03d}' for i in range(n_samples)]
    disease_status = ['Healthy'] * 50 + ['Disease'] * 50
    batch = ['Batch1'] * 50 + ['Batch2'] * 50  # Samples split into 2 batches
    
    metadata = pd.DataFrame({
        'Sample_ID': sample_ids,
        'Disease_Status': disease_status,
        'Batch': batch
    })
    
    # Create protein names
    protein_names = [f'Protein_{i:03d}' for i in range(n_proteins)]
    
    # Generate base expression data
    # Different proteins have different mean expression levels (simulating abundance)
    protein_means = np.random.uniform(5, 15, n_proteins)  # Different baseline levels
    protein_stds = np.random.uniform(0.5, 2.5, n_proteins)  # Different variability
    
    data = np.zeros((n_samples, n_proteins))
    
    for i in range(n_proteins):
        data[:, i] = np.random.normal(protein_means[i], protein_stds[i], n_samples)
    
    # Add disease signal to specific proteins (first 50 proteins are disease-related)
    disease_proteins = 50
    for i in range(disease_proteins):
        # Disease samples (last 50) have higher expression of these proteins
        effect_size = np.random.uniform(1.5, 3.0)  # Fold change
        data[50:, i] += effect_size
    
    # Add batch effect (Batch2 has systematic shift)
    batch_effect_size = 1.5
    data[50:, :] += batch_effect_size  # Batch2 samples have higher overall signal
    
    # Create outlier samples (2-3 samples with unusual patterns)
    outlier_indices = [10, 35, 87]
    for idx in outlier_indices:
        # Add random large deviations
        data[idx, :] += np.random.normal(0, 3, n_proteins)
    
    # Create DataFrame
    df = pd.DataFrame(data, index=sample_ids, columns=protein_names)
    
    # Introduce missing values (15-20% missing)
    # More likely to be missing for low-abundance proteins
    missing_rate = 0.175  # 17.5% missing
    
    for i, protein in enumerate(protein_names):
        # Lower abundance proteins have more missing values
        protein_missing_rate = missing_rate * (1 + (15 - protein_means[i]) / 10)
        protein_missing_rate = np.clip(protein_missing_rate, 0.05, 0.40)
        
        # Randomly set values to NaN
        missing_mask = np.random.random(n_samples) < protein_missing_rate
        df.loc[missing_mask, protein] = np.nan
    
    # Make some proteins very sparse (>50% missing) - these should be filtered out
    very_sparse_proteins = np.random.choice(protein_names, 20, replace=False)
    for protein in very_sparse_proteins:
        missing_mask = np.random.random(n_samples) < 0.65  # 65% missing
        df.loc[missing_mask, protein] = np.nan
    
    # Make some samples very sparse (>30% missing) - these should be filtered out
    very_sparse_samples = [sample_ids[5], sample_ids[73]]
    for sample in very_sparse_samples:
        missing_mask = np.random.choice([True, False], n_proteins, p=[0.45, 0.55])
        df.loc[sample, missing_mask] = np.nan
    
    return df, metadata

# Generate the data
print("Generating proteomics dataset...")
df_raw, metadata = generate_proteomics_data()

print(f"\n✓ Dataset generated successfully!")
print(f"  - Samples: {df_raw.shape[0]}")
print(f"  - Proteins: {df_raw.shape[1]}")
print(f"  - Missing values: {df_raw.isna().sum().sum()} ({100*df_raw.isna().sum().sum()/(df_raw.shape[0]*df_raw.shape[1]):.1f}%)")
print(f"\nMetadata:")
print(metadata.head())
print(f"\nExpression data (first 5 samples, first 5 proteins):")
print(df_raw.iloc[:5, :5])

## 3. Data Exploration

Before cleaning, let's understand our data quality issues.

In [ ]:
# Basic statistics
print("=" * 60)
print("DATA QUALITY ASSESSMENT")
print("=" * 60)

# Missing values per sample
missing_per_sample = df_raw.isna().sum(axis=1)
print(f"\nMissing values per sample:")
print(f"  Mean: {missing_per_sample.mean():.1f} proteins ({100*missing_per_sample.mean()/df_raw.shape[1]:.1f}%)")
print(f"  Max: {missing_per_sample.max()} proteins ({100*missing_per_sample.max()/df_raw.shape[1]:.1f}%)")
print(f"  Samples with >30% missing: {(missing_per_sample > 0.3*df_raw.shape[1]).sum()}")

# Missing values per protein
missing_per_protein = df_raw.isna().sum(axis=0)
print(f"\nMissing values per protein:")
print(f"  Mean: {missing_per_protein.mean():.1f} samples ({100*missing_per_protein.mean()/df_raw.shape[0]:.1f}%)")
print(f"  Max: {missing_per_protein.max()} samples ({100*missing_per_protein.max()/df_raw.shape[0]:.1f}%)")
print(f"  Proteins with >50% missing: {(missing_per_protein > 0.5*df_raw.shape[0]).sum()}")

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribution of expression values
ax = axes[0, 0]
sample_data = df_raw.iloc[0, :].dropna()  # First sample
ax.hist(sample_data, bins=50, edgecolor='black', alpha=0.7)
ax.set_xlabel('Expression Value')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Protein Expression (Sample 1)')
ax.axvline(sample_data.mean(), color='red', linestyle='--', label=f'Mean: {sample_data.mean():.2f}')
ax.legend()

# 2. Missing values per sample
ax = axes[0, 1]
ax.bar(range(len(missing_per_sample)), missing_per_sample.values, alpha=0.7)
ax.axhline(y=0.3*df_raw.shape[1], color='red', linestyle='--', label='30% threshold')
ax.set_xlabel('Sample Index')
ax.set_ylabel('Number of Missing Proteins')
ax.set_title('Missing Values per Sample')
ax.legend()

# 3. Missing values per protein (histogram)
ax = axes[1, 0]
ax.hist(missing_per_protein.values, bins=30, edgecolor='black', alpha=0.7)
ax.axvline(x=0.5*df_raw.shape[0], color='red', linestyle='--', label='50% threshold')
ax.set_xlabel('Number of Missing Samples')
ax.set_ylabel('Number of Proteins')
ax.set_title('Distribution of Missing Values Across Proteins')
ax.legend()

# 4. Expression range across proteins (boxplot of means)
ax = axes[1, 1]
protein_means = df_raw.mean(axis=0)
ax.hist(protein_means, bins=50, edgecolor='black', alpha=0.7)
ax.set_xlabel('Mean Expression Level')
ax.set_ylabel('Number of Proteins')
ax.set_title('Distribution of Mean Protein Expression Levels\n(Shows Dynamic Range)')

plt.tight_layout()
plt.show()

print("\n✓ Exploration complete! Notice the data quality issues we need to address.")

## 4. Data Cleaning Pipeline

### Why Data Cleaning Matters

In proteomics, **garbage in = garbage out**. We need to:
1. Remove unreliable measurements (too many missing values)
2. Handle remaining missing values appropriately
3. Detect outlier samples that might be technical failures

### The Cleaning Strategy
1. **Filter proteins** with >50% missing (unreliable)
2. **Filter samples** with >30% missing (poor quality)
3. **Impute** remaining missing values
4. **Detect outliers** using PCA
5. **Decide** whether to remove outliers

Let's go step by step!

### Step 1: Visualize Missingness Pattern

In [ ]:
# Create a missingness heatmap (sample every 5th protein to make it visible)
fig, ax = plt.subplots(figsize=(15, 8))

# Sample proteins for visualization
protein_subset = df_raw.columns[::10]  # Every 10th protein
missing_matrix = df_raw[protein_subset].isna().astype(int)

sns.heatmap(missing_matrix.T, cmap='RdYlGn_r', cbar_kws={'label': 'Missing (1) vs Present (0)'},
            yticklabels=True, xticklabels=False, ax=ax)
ax.set_xlabel('Samples')
ax.set_ylabel('Proteins (every 10th shown)')
ax.set_title('Missingness Pattern\n(Red = Missing, Green = Present)')

plt.tight_layout()
plt.show()

print("Notice:")
print("  - Some proteins (rows) are mostly red = sparse proteins to filter")
print("  - Some samples (columns) have many red streaks = poor quality samples")
print("  - Missing values are NOT random - this is realistic!")

### Step 2: Filter Proteins with >50% Missing Values

**Why?** Proteins with too many missing values are unreliable. Imputing >50% of the data would introduce too much noise.

In [ ]:
# Calculate missing percentage per protein
missing_pct_protein = df_raw.isna().sum(axis=0) / df_raw.shape[0]

# Filter proteins
threshold_protein = 0.50
proteins_to_keep = missing_pct_protein[missing_pct_protein <= threshold_protein].index
df_filtered_proteins = df_raw[proteins_to_keep].copy()

print(f"Protein Filtering (>{threshold_protein*100:.0f}% missing):")
print(f"  - Original proteins: {df_raw.shape[1]}")
print(f"  - Proteins removed: {df_raw.shape[1] - df_filtered_proteins.shape[1]}")
print(f"  - Proteins retained: {df_filtered_proteins.shape[1]}")
print(f"  - Retention rate: {100*df_filtered_proteins.shape[1]/df_raw.shape[1]:.1f}%")

### Step 3: Filter Samples with >30% Missing Values

**Why?** Samples with too many missing values likely had technical problems during processing.

In [ ]:
# Calculate missing percentage per sample
missing_pct_sample = df_filtered_proteins.isna().sum(axis=1) / df_filtered_proteins.shape[1]

# Filter samples
threshold_sample = 0.30
samples_to_keep = missing_pct_sample[missing_pct_sample <= threshold_sample].index
df_filtered_both = df_filtered_proteins.loc[samples_to_keep].copy()

# Update metadata
metadata_filtered = metadata[metadata['Sample_ID'].isin(samples_to_keep)].copy()
metadata_filtered.reset_index(drop=True, inplace=True)

print(f"Sample Filtering (>{threshold_sample*100:.0f}% missing):")
print(f"  - Original samples: {df_filtered_proteins.shape[0]}")
print(f"  - Samples removed: {df_filtered_proteins.shape[0] - df_filtered_both.shape[0]}")
print(f"  - Samples retained: {df_filtered_both.shape[0]}")
print(f"  - Retention rate: {100*df_filtered_both.shape[0]/df_filtered_proteins.shape[0]:.1f}%")

print(f"\nAfter filtering:")
print(f"  - Dataset shape: {df_filtered_both.shape}")
print(f"  - Remaining missing values: {df_filtered_both.isna().sum().sum()} ({100*df_filtered_both.isna().sum().sum()/(df_filtered_both.shape[0]*df_filtered_both.shape[1]):.1f}%)")

### Step 4: Impute Remaining Missing Values

**Imputation Methods:**
1. **Mean imputation**: Replace missing with column mean (simple, but ignores sample similarity)
2. **Median imputation**: More robust to outliers than mean
3. **KNN imputation**: Use k-nearest neighbors (considers sample similarity)

**Which to use?** KNN is usually best for proteomics, but computationally expensive. Let's compare!

In [ ]:
def impute_data(df, method='mean', n_neighbors=5):
    """
    Impute missing values using specified method.
    
    Parameters:
    -----------
    df : DataFrame
    method : str, one of 'mean', 'median', 'knn'
    n_neighbors : int, for KNN imputation
    
    Returns:
    --------
    df_imputed : DataFrame with no missing values
    """
    if method == 'mean':
        imputer = SimpleImputer(strategy='mean')
    elif method == 'median':
        imputer = SimpleImputer(strategy='median')
    elif method == 'knn':
        imputer = KNNImputer(n_neighbors=n_neighbors)
    else:
        raise ValueError(f"Unknown method: {method}")
    
    data_imputed = imputer.fit_transform(df)
    df_imputed = pd.DataFrame(data_imputed, index=df.index, columns=df.columns)
    
    return df_imputed

# Try all three methods
print("Imputing missing values with different methods...\n")

df_imputed_mean = impute_data(df_filtered_both, method='mean')
print("✓ Mean imputation complete")

df_imputed_median = impute_data(df_filtered_both, method='median')
print("✓ Median imputation complete")

df_imputed_knn = impute_data(df_filtered_both, method='knn', n_neighbors=5)
print("✓ KNN imputation complete")

# Verify no missing values
print(f"\nMissing values after imputation:")
print(f"  - Mean: {df_imputed_mean.isna().sum().sum()}")
print(f"  - Median: {df_imputed_median.isna().sum().sum()}")
print(f"  - KNN: {df_imputed_knn.isna().sum().sum()}")

# Compare imputation methods by looking at variance introduced
print(f"\nMean expression level (should be similar):")
print(f"  - Mean imputation: {df_imputed_mean.values.mean():.3f}")
print(f"  - Median imputation: {df_imputed_median.values.mean():.3f}")
print(f"  - KNN imputation: {df_imputed_knn.values.mean():.3f}")

# We'll use KNN for subsequent analysis (generally best practice)
df_imputed = df_imputed_knn.copy()
print("\n✓ Using KNN imputation for subsequent analysis")

In [ ]:
# TODO: EXERCISE 1 - Try a different imputation method or parameter
# 
# Try these experiments:
# 1. Change n_neighbors in KNN imputation (try 3, 10, 15)
# 2. Compare results using PCA plots (see Step 5 below for PCA code)
# 3. Which method seems to preserve the data structure best?
#
# Your code here:
# df_imputed_knn_k3 = impute_data(df_filtered_both, method='knn', n_neighbors=3)
# ... compare with PCA ...


### Step 5: Detect Outlier Samples Using PCA

**Principal Component Analysis (PCA)** reduces our 500-dimensional data to 2D for visualization. 

**Outliers** will appear far from the main cluster. These could be:
- Technical failures (sample degradation, processing errors)
- Interesting biological samples (rare subtypes)

**Decision:** Removing outliers improves model performance but loses information. We'll identify them, then decide.

In [ ]:
# Perform PCA
pca = PCA(n_components=2)
pca_coords = pca.fit_transform(df_imputed)

# Create PCA DataFrame
pca_df = pd.DataFrame(pca_coords, columns=['PC1', 'PC2'], index=df_imputed.index)
pca_df = pca_df.merge(metadata_filtered, left_index=True, right_on='Sample_ID')

# Detect outliers using z-score method (>3 SD from mean)
from scipy.stats import zscore
z_scores = np.abs(zscore(pca_coords, axis=0))
outlier_threshold = 3
outliers = (z_scores > outlier_threshold).any(axis=1)

pca_df['Is_Outlier'] = outliers

print(f"Outlier Detection:")
print(f"  - Outliers detected: {outliers.sum()}")
print(f"  - Outlier samples: {', '.join(pca_df[pca_df['Is_Outlier']]['Sample_ID'].values)}")

# Visualize PCA with outliers highlighted
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Colored by disease status
ax = axes[0]
for disease in ['Healthy', 'Disease']:
    mask = pca_df['Disease_Status'] == disease
    ax.scatter(pca_df[mask]['PC1'], pca_df[mask]['PC2'], 
               label=disease, alpha=0.6, s=100)

# Highlight outliers
outlier_data = pca_df[pca_df['Is_Outlier']]
ax.scatter(outlier_data['PC1'], outlier_data['PC2'], 
           color='red', s=200, marker='x', linewidths=3, label='Outliers')

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)')
ax.set_title('PCA: Outlier Detection\n(Red X = Outliers)')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Colored by batch (to see batch effects)
ax = axes[1]
for batch in ['Batch1', 'Batch2']:
    mask = pca_df['Batch'] == batch
    ax.scatter(pca_df[mask]['PC1'], pca_df[mask]['PC2'], 
               label=batch, alpha=0.6, s=100)

ax.scatter(outlier_data['PC1'], outlier_data['PC2'], 
           color='red', s=200, marker='x', linewidths=3, label='Outliers')

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)')
ax.set_title('PCA: Batch Effect Visible\n(Notice batch separation!)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("  - Left plot: Can you see disease separation?")
print("  - Right plot: Notice how batches cluster separately?")
print("  - This batch effect needs correction!")

### Step 6: (Optional) Remove Outliers

**Decision time!** Should we remove outliers?

**Pros:** Better model performance, cleaner patterns  
**Cons:** Lose potentially interesting samples, reduce sample size

For this exercise, we'll keep them to maintain sample size, but you can experiment!

In [ ]:
# Option to remove outliers (set to False to keep them)
REMOVE_OUTLIERS = False

if REMOVE_OUTLIERS:
    # Remove outliers
    non_outlier_samples = pca_df[~pca_df['Is_Outlier']]['Sample_ID'].values
    df_cleaned = df_imputed.loc[non_outlier_samples].copy()
    metadata_cleaned = metadata_filtered[metadata_filtered['Sample_ID'].isin(non_outlier_samples)].copy()
    print(f"✓ Outliers removed. Remaining samples: {df_cleaned.shape[0]}")
else:
    df_cleaned = df_imputed.copy()
    metadata_cleaned = metadata_filtered.copy()
    print(f"✓ Outliers kept. Total samples: {df_cleaned.shape[0]}")

print(f"\nFinal cleaned dataset:")
print(f"  - Samples: {df_cleaned.shape[0]}")
print(f"  - Proteins: {df_cleaned.shape[1]}")
print(f"  - Missing values: {df_cleaned.isna().sum().sum()}")
print(f"  - Disease distribution: {metadata_cleaned['Disease_Status'].value_counts().to_dict()}")

In [ ]:
# TODO: EXERCISE 2 - Experiment with different filtering thresholds
#
# Try these experiments:
# 1. Change protein filtering threshold (try 0.3, 0.4, 0.6)
# 2. Change sample filtering threshold (try 0.2, 0.4)
# 3. How does this affect your final dataset size?
# 4. What's the trade-off between data quality and sample size?
#
# Your code here:
# threshold_protein = 0.40
# ... repeat filtering steps ...


## 5. Normalization Methods

### Why Normalize?

Proteins have vastly different expression levels (some are 1000x more abundant than others). Without normalization:
- High-abundance proteins dominate ML algorithms
- Sample-to-sample technical variation obscures biology
- Distance-based methods (PCA, SVM) perform poorly

### Methods We'll Compare:

1. **Quantile Normalization**: Makes sample distributions identical (assumes most proteins don't change)
2. **Z-score (per protein)**: Centers and scales each protein to mean=0, std=1 (makes proteins comparable)
3. **Log2 + Scaling**: Log transform to reduce dynamic range, then scale (common in genomics)

### When to Use Each:
- **Quantile**: When you expect most proteins unchanged between groups
- **Z-score**: When proteins have different dynamic ranges (most common)
- **Log2**: When data is log-normally distributed (common in RNA-seq, proteomics)

Let's implement and compare!

In [ ]:
def quantile_normalize(df):
    """
    Quantile normalization: make all sample distributions identical.
    """
    # Sort each column
    sorted_df = pd.DataFrame(np.sort(df.values, axis=0), 
                             index=df.index, columns=df.columns)
    # Calculate mean of each rank
    rank_means = sorted_df.mean(axis=1)
    # Get ranks of original data
    ranks = df.rank(method='min').astype(int) - 1
    # Replace with mean values
    normalized = ranks.apply(lambda x: rank_means.iloc[x].values, axis=0)
    return pd.DataFrame(normalized, index=df.index, columns=df.columns)

def zscore_normalize(df):
    """
    Z-score normalization per protein: mean=0, std=1.
    """
    return df.apply(lambda x: (x - x.mean()) / x.std(), axis=0)

def log2_normalize(df):
    """
    Log2 transformation + standardization.
    """
    # Add small constant to avoid log(0)
    df_shifted = df - df.min().min() + 1
    df_log = np.log2(df_shifted)
    # Standardize
    scaler = StandardScaler()
    df_normalized = pd.DataFrame(scaler.fit_transform(df_log),
                                  index=df.index, columns=df.columns)
    return df_normalized

print("Applying normalization methods...\n")

# Apply all normalization methods
df_quantile = quantile_normalize(df_cleaned)
print("✓ Quantile normalization complete")

df_zscore = zscore_normalize(df_cleaned)
print("✓ Z-score normalization complete")

df_log2 = log2_normalize(df_cleaned)
print("✓ Log2 normalization complete")

print("\n✓ All normalization methods applied!")

### Visualizing Normalization Effects

Let's see how each method changes the data distribution.

In [ ]:
# Compare normalization methods
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

datasets = [
    ('Raw', df_cleaned),
    ('Quantile', df_quantile),
    ('Z-score', df_zscore),
    ('Log2', df_log2)
]

# Row 1: Boxplots (sample distributions)
for idx, (name, data) in enumerate(datasets):
    ax = axes[0, idx]
    # Sample 10 random samples for clarity
    sample_subset = data.iloc[:10, :]
    ax.boxplot([sample_subset.iloc[i, :] for i in range(10)], 
               labels=[f'S{i+1}' for i in range(10)])
    ax.set_title(f'{name}\nBoxplot Across Samples')
    ax.set_xlabel('Sample')
    ax.set_ylabel('Expression')
    ax.tick_params(axis='x', rotation=45)

# Row 2: Histograms (overall distribution)
for idx, (name, data) in enumerate(datasets):
    ax = axes[1, idx]
    # Flatten all values
    all_values = data.values.flatten()
    ax.hist(all_values, bins=50, edgecolor='black', alpha=0.7)
    ax.set_title(f'{name}\nOverall Distribution')
    ax.set_xlabel('Expression Value')
    ax.set_ylabel('Frequency')
    ax.axvline(np.mean(all_values), color='red', linestyle='--', 
               label=f'Mean: {np.mean(all_values):.2f}')
    ax.legend()

plt.tight_layout()
plt.show()

print("\nObservations:")
print("  - Raw: Wide range, different medians per sample")
print("  - Quantile: All samples have identical distributions")
print("  - Z-score: Centered at 0, same scale across proteins")
print("  - Log2: Compressed dynamic range, normalized scale")

## 6. Batch Effect Correction

### What are Batch Effects?

**Batch effects** = systematic technical differences between groups of samples processed together.

In our data:
- Batch 1: Samples 0-49 (processed on Day 1)
- Batch 2: Samples 50-99 (processed on Day 2)

**Problem:** Batch effects can be LARGER than biological effects!

**Solution:** Statistical correction to remove batch effects while preserving biology.

### Methods:
1. **Mean centering**: Subtract batch-specific mean (simple, fast)
2. **ComBat**: Empirical Bayes method (gold standard, but needs extra library)

We'll use mean centering for simplicity.

In [ ]:
# Visualize batch effect BEFORE correction
def visualize_batch_effect(data, metadata, title_prefix=""):
    """
    Visualize batch effects using PCA.
    """
    pca = PCA(n_components=2)
    pca_coords = pca.fit_transform(data)
    
    pca_df = pd.DataFrame(pca_coords, columns=['PC1', 'PC2'])
    pca_df['Batch'] = metadata['Batch'].values
    pca_df['Disease'] = metadata['Disease_Status'].values
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot by batch
    ax = axes[0]
    for batch in ['Batch1', 'Batch2']:
        mask = pca_df['Batch'] == batch
        ax.scatter(pca_df[mask]['PC1'], pca_df[mask]['PC2'], 
                   label=batch, alpha=0.6, s=100)
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
    ax.set_title(f'{title_prefix}PCA by Batch')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot by disease
    ax = axes[1]
    for disease in ['Healthy', 'Disease']:
        mask = pca_df['Disease'] == disease
        ax.scatter(pca_df[mask]['PC1'], pca_df[mask]['PC2'], 
                   label=disease, alpha=0.6, s=100)
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
    ax.set_title(f'{title_prefix}PCA by Disease Status')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return pca.explained_variance_ratio_

# Before correction
print("BEFORE Batch Correction:")
var_before = visualize_batch_effect(df_zscore, metadata_cleaned, "BEFORE Correction: ")
print("\nNotice: Batch separation is visible in PC1!")
print("This means batch effects are a major source of variance.")

In [ ]:
def batch_correct_mean_centering(data, metadata):
    """
    Simple batch correction: subtract batch-specific mean for each protein.
    """
    data_corrected = data.copy()
    
    for batch in metadata['Batch'].unique():
        # Get samples in this batch
        batch_samples = metadata[metadata['Batch'] == batch]['Sample_ID'].values
        
        # Calculate batch mean for each protein
        batch_mean = data_corrected.loc[batch_samples].mean(axis=0)
        
        # Subtract batch mean
        data_corrected.loc[batch_samples] -= batch_mean
    
    return data_corrected

# Apply batch correction
df_batch_corrected = batch_correct_mean_centering(df_zscore, metadata_cleaned)

print("\nAFTER Batch Correction:")
var_after = visualize_batch_effect(df_batch_corrected, metadata_cleaned, "AFTER Correction: ")

print("\nNotice: Batch separation reduced!")
print("Disease signal is now more visible.")
print(f"\nVariance explained by PC1: Before={var_before[0]*100:.1f}%, After={var_after[0]*100:.1f}%")

## 7. Machine Learning: Disease Prediction

### The Task

Can we predict disease status from protein expression?

### Models We'll Try:
1. **Logistic Regression**: Linear model, interpretable, fast
2. **Random Forest**: Ensemble of decision trees, handles non-linearity
3. **SVM**: Support Vector Machine, good for high-dimensional data

### Evaluation Strategy:
- **Train/test split**: 70/30, stratified (maintain class balance)
- **Cross-validation**: 5-fold CV on training set
- **Metrics**: Accuracy, ROC-AUC, confusion matrix

### Hypothesis:
**Proper preprocessing should improve all models!**

In [ ]:
def prepare_ml_data(data, metadata, test_size=0.3, random_state=42):
    """
    Prepare data for machine learning.
    
    Returns:
    --------
    X_train, X_test, y_train, y_test
    """
    # Features (protein expression)
    X = data.values
    
    # Target (disease status: 0=Healthy, 1=Disease)
    y = (metadata['Disease_Status'] == 'Disease').astype(int).values
    
    # Stratified split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    return X_train, X_test, y_train, y_test

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """
    Train and evaluate a model.
    """
    # Train
    model.fit(X_train, y_train)
    
    # Cross-validation on training set
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    
    # Test set predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Metrics
    test_auc = roc_auc_score(y_test, y_pred_proba)
    
    return {
        'model': model,
        'model_name': model_name,
        'cv_auc_mean': cv_scores.mean(),
        'cv_auc_std': cv_scores.std(),
        'test_auc': test_auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'y_test': y_test
    }

# Prepare data (using batch-corrected, normalized data)
X_train, X_test, y_train, y_test = prepare_ml_data(df_batch_corrected, metadata_cleaned)

print("Data prepared for machine learning:")
print(f"  - Training samples: {X_train.shape[0]} (Healthy: {(y_train==0).sum()}, Disease: {(y_train==1).sum()})")
print(f"  - Test samples: {X_test.shape[0]} (Healthy: {(y_test==0).sum()}, Disease: {(y_test==1).sum()})")
print(f"  - Features: {X_train.shape[1]} proteins")

### Training Models

In [ ]:
# Define models
models = [
    (LogisticRegression(max_iter=1000, random_state=42), "Logistic Regression"),
    (RandomForestClassifier(n_estimators=100, random_state=42), "Random Forest"),
    (SVC(kernel='rbf', probability=True, random_state=42), "SVM (RBF)")
]

# Train and evaluate all models
results = []

print("Training models...\n")
for model, name in models:
    print(f"Training {name}...")
    result = evaluate_model(model, X_train, X_test, y_train, y_test, name)
    results.append(result)
    print(f"  ✓ CV AUC: {result['cv_auc_mean']:.3f} ± {result['cv_auc_std']:.3f}")
    print(f"  ✓ Test AUC: {result['test_auc']:.3f}\n")

print("✓ All models trained!")

### Model Performance Comparison

In [ ]:
# Create comprehensive results visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. ROC Curves
ax = axes[0, 0]
for result in results:
    fpr, tpr, _ = roc_curve(result['y_test'], result['y_pred_proba'])
    ax.plot(fpr, tpr, label=f"{result['model_name']} (AUC={result['test_auc']:.3f})")
ax.plot([0, 1], [0, 1], 'k--', label='Random')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves (Test Set)')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Performance Bar Chart
ax = axes[0, 1]
model_names = [r['model_name'] for r in results]
cv_aucs = [r['cv_auc_mean'] for r in results]
test_aucs = [r['test_auc'] for r in results]

x = np.arange(len(model_names))
width = 0.35

ax.bar(x - width/2, cv_aucs, width, label='CV AUC (Train)', alpha=0.8)
ax.bar(x + width/2, test_aucs, width, label='Test AUC', alpha=0.8)
ax.set_xlabel('Model')
ax.set_ylabel('AUC Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(model_names, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0.5, 1.0])

# 3-4. Confusion Matrices for top 2 models
for idx, result in enumerate(results[:2]):
    ax = axes[1, idx]
    cm = confusion_matrix(result['y_test'], result['y_pred'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Healthy', 'Disease'],
                yticklabels=['Healthy', 'Disease'])
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')
    ax.set_title(f'Confusion Matrix: {result["model_name"]}')

plt.tight_layout()
plt.show()

# Print detailed results
print("\n" + "="*60)
print("DETAILED MODEL PERFORMANCE")
print("="*60)
for result in results:
    print(f"\n{result['model_name']}:")
    print(f"  Cross-Validation AUC: {result['cv_auc_mean']:.3f} ± {result['cv_auc_std']:.3f}")
    print(f"  Test Set AUC: {result['test_auc']:.3f}")
    print("\n  Classification Report:")
    print(classification_report(result['y_test'], result['y_pred'], 
                                target_names=['Healthy', 'Disease']))

## 8. Feature Importance: Which Proteins Matter Most?

Understanding **which proteins** drive the predictions helps with:
- Biological interpretation
- Biomarker discovery
- Validating model decisions

We'll examine feature importance from Random Forest (other models have different methods).

In [ ]:
# Get Random Forest model
rf_result = [r for r in results if 'Random Forest' in r['model_name']][0]
rf_model = rf_result['model']

# Get feature importances
importances = rf_model.feature_importances_
protein_names = df_batch_corrected.columns

# Create DataFrame
importance_df = pd.DataFrame({
    'Protein': protein_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

# Display top 20
print("Top 20 Most Important Proteins:")
print(importance_df.head(20).to_string(index=False))

# Visualize top 10
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Bar chart of top 15
ax = axes[0]
top_features = importance_df.head(15)
ax.barh(range(len(top_features)), top_features['Importance'].values, alpha=0.7)
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['Protein'].values)
ax.set_xlabel('Importance Score')
ax.set_title('Top 15 Most Important Proteins')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

# Plot 2: Cumulative importance
ax = axes[1]
cumulative_importance = np.cumsum(importance_df['Importance'].values)
ax.plot(range(len(cumulative_importance)), cumulative_importance)
ax.axhline(y=0.9, color='red', linestyle='--', label='90% of importance')
ax.set_xlabel('Number of Proteins')
ax.set_ylabel('Cumulative Importance')
ax.set_title('Cumulative Feature Importance')
ax.legend()
ax.grid(True, alpha=0.3)

# Find how many features explain 90% of importance
n_features_90 = np.argmax(cumulative_importance >= 0.9) + 1
ax.axvline(x=n_features_90, color='red', linestyle='--', alpha=0.5)
ax.text(n_features_90, 0.5, f'{n_features_90} proteins\nexplain 90%', 
        ha='left', va='center')

plt.tight_layout()
plt.show()

print(f"\n✓ {n_features_90} proteins explain 90% of the model's predictive power!")
print(f"  (out of {len(protein_names)} total proteins)")
print("\nThis suggests we could build a simpler model with fewer proteins!")

In [ ]:
# TODO: EXERCISE 5 - Visualize top 10 most important proteins
#
# Create boxplots showing expression levels of the top 10 proteins,
# separated by disease status (Healthy vs Disease)
#
# Hints:
# 1. Get top 10 protein names from importance_df
# 2. Extract their expression values from df_batch_corrected
# 3. Use seaborn.boxplot() or matplotlib to create side-by-side boxplots
# 4. Color by disease status using metadata_cleaned
#
# Your code here:
# top_10_proteins = importance_df.head(10)['Protein'].values
# ...


## 9. Student Exercises

Now it's your turn to experiment! Try these exercises to deepen your understanding.

### Exercise 3: Feature Selection Before ML

Instead of using all proteins, try selecting only the most informative ones before training.

In [ ]:
# TODO: EXERCISE 3 - Feature Selection
#
# Try these approaches:
# 1. Use only top 50 proteins by variance (most variable across samples)
# 2. Use SelectKBest from sklearn.feature_selection
# 3. Compare model performance with all features vs selected features
#
# Hints:
# - Calculate variance: df_batch_corrected.var(axis=0)
# - SelectKBest: from sklearn.feature_selection import SelectKBest, f_classif
# - Retrain models on reduced feature set
#
# Your code here:
# from sklearn.feature_selection import SelectKBest, f_classif
# ...


### Exercise 4: Different Train/Test Splits

How does the train/test split ratio affect model performance?

In [ ]:
# TODO: EXERCISE 4 - Experiment with train/test splits
#
# Try different split ratios and compare results:
# 1. 80/20 split
# 2. 60/40 split  
# 3. 50/50 split
#
# Questions to explore:
# - How does test set performance change with less training data?
# - Does cross-validation AUC differ significantly?
# - What's the optimal split for this dataset size?
#
# Your code here:
# test_sizes = [0.2, 0.4, 0.5]
# for test_size in test_sizes:
#     X_train, X_test, y_train, y_test = prepare_ml_data(df_batch_corrected, metadata_cleaned, test_size=test_size)
#     ...


## 10. Impact Analysis: Why Preprocessing Matters

### The Big Question: 
**What happens if we skip preprocessing steps?**

Let's compare model performance with different preprocessing strategies:
1. **No preprocessing**: Raw data with simple imputation
2. **Only normalization**: Skip batch correction
3. **Full pipeline**: All preprocessing steps

This will show you **why each step matters**!

In [ ]:
print("Comparing preprocessing strategies...\n")
print("This may take a minute...\n")

# Strategy 1: Minimal preprocessing (just impute, no normalization, no batch correction)
df_minimal = df_filtered_both.copy()
df_minimal = impute_data(df_minimal, method='mean')  # Simple mean imputation
X_train_min, X_test_min, y_train_min, y_test_min = prepare_ml_data(df_minimal, metadata_cleaned)

# Strategy 2: Normalization only (no batch correction)
df_norm_only = zscore_normalize(df_cleaned)
X_train_norm, X_test_norm, y_train_norm, y_test_norm = prepare_ml_data(df_norm_only, metadata_cleaned)

# Strategy 3: Full pipeline (normalization + batch correction)
X_train_full, X_test_full, y_train_full, y_test_full = X_train, X_test, y_train, y_test  # Already computed

# Test all strategies with Random Forest (typically best performing)
strategies = [
    ("Minimal (Raw + Mean Impute)", X_train_min, X_test_min, y_train_min, y_test_min),
    ("Normalization Only", X_train_norm, X_test_norm, y_train_norm, y_test_norm),
    ("Full Pipeline", X_train_full, X_test_full, y_train_full, y_test_full)
]

strategy_results = []

for strategy_name, X_tr, X_te, y_tr, y_te in strategies:
    print(f"Testing: {strategy_name}...")
    
    # Train Random Forest
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    result = evaluate_model(rf, X_tr, X_te, y_tr, y_te, strategy_name)
    strategy_results.append(result)
    
    print(f"  ✓ Test AUC: {result['test_auc']:.3f}\n")

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: AUC comparison
ax = axes[0]
strategy_names = [r['model_name'] for r in strategy_results]
cv_aucs = [r['cv_auc_mean'] for r in strategy_results]
test_aucs = [r['test_auc'] for r in strategy_results]

x = np.arange(len(strategy_names))
width = 0.35

bars1 = ax.bar(x - width/2, cv_aucs, width, label='CV AUC', alpha=0.8)
bars2 = ax.bar(x + width/2, test_aucs, width, label='Test AUC', alpha=0.8)

ax.set_ylabel('AUC Score')
ax.set_title('Impact of Preprocessing on Model Performance\n(Random Forest)')
ax.set_xticks(x)
ax.set_xticklabels(strategy_names, rotation=15, ha='right')
ax.legend()
ax.set_ylim([0.5, 1.0])
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

# Plot 2: ROC curves
ax = axes[1]
for result in strategy_results:
    fpr, tpr, _ = roc_curve(result['y_test'], result['y_pred_proba'])
    ax.plot(fpr, tpr, label=f"{result['model_name']} (AUC={result['test_auc']:.3f})", linewidth=2)
ax.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves: Preprocessing Comparison')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*70)
print("PREPROCESSING IMPACT SUMMARY")
print("="*70)

baseline_auc = strategy_results[0]['test_auc']
for result in strategy_results:
    improvement = result['test_auc'] - baseline_auc
    print(f"\n{result['model_name']}:")
    print(f"  Test AUC: {result['test_auc']:.3f}")
    print(f"  Improvement over baseline: {improvement:+.3f} ({100*improvement/baseline_auc:+.1f}%)")

print("\n" + "="*70)
print("KEY TAKEAWAYS:")
print("="*70)
print("1. Raw data performs poorly due to:")
print("   - Technical variation overwhelming biological signal")
print("   - Different protein scales confusing the model")
print("   - Batch effects creating false patterns")
print("\n2. Normalization helps by:")
print("   - Making proteins comparable (same scale)")
print("   - Reducing technical noise")
print("\n3. Batch correction is crucial:")
print("   - Removes systematic technical artifacts")
print("   - Reveals true biological differences")
print("\n4. Full preprocessing pipeline maximizes:")
print("   - Predictive accuracy")
print("   - Biological interpretability")
print("   - Reproducibility across batches")
print("\n⚠️  NEVER skip preprocessing in real proteomics analysis!")

## 11. Summary and Recommended Pipeline

### What We Learned

1. **Data Quality Issues** in proteomics are real and significant:
   - Missing values (15-20% typical)
   - Batch effects (can be larger than biology!)
   - Outlier samples
   - Different protein abundance levels

2. **Preprocessing Steps Matter** - each improves results:
   - Filtering unreliable proteins/samples
   - KNN imputation > mean imputation
   - Z-score normalization for cross-protein comparison
   - Batch effect correction is critical

3. **Model Selection**:
   - Random Forest typically performs best
   - All models benefit from proper preprocessing
   - Feature importance reveals biological insights

### Recommended Pipeline for Proteomics Data

```
Raw Data
    ↓
Filter proteins (>50% missing)
    ↓
Filter samples (>30% missing)
    ↓
KNN imputation (k=5)
    ↓
Z-score normalization
    ↓
Batch effect correction
    ↓
PCA visualization (QC check)
    ↓
Machine Learning
    ↓
Feature importance analysis
```

### Best Practices

1. **Always visualize** before and after each step
2. **Use cross-validation** to avoid overfitting
3. **Check for batch effects** with PCA
4. **Feature importance** helps biological interpretation
5. **Compare multiple models** - no single best approach

### Further Exploration

Try these on real proteomics datasets:
- **Advanced imputation**: MICE, missForest
- **Advanced normalization**: ComBat, limma
- **Feature selection**: LASSO, elastic net
- **Deep learning**: neural networks for complex patterns
- **Pathway analysis**: connect proteins to biological processes

---

## Congratulations! 🎉

You've completed a full proteomics machine learning pipeline. These skills apply to:
- RNA-seq analysis
- Metabolomics
- Clinical biomarker discovery
- Any high-dimensional biological data

**Remember:** In bioinformatics, **data quality determines results**. 
Spend time on preprocessing - it's not glamorous, but it's essential!

---

## Additional Resources

### Python Libraries for Proteomics:
- **pyteomics**: Proteomics data parsing
- **biopython**: General bioinformatics
- **scanpy**: Single-cell analysis (similar workflows)

### Further Reading:
- "Normalization methods for proteomics data" - review papers
- ComBat batch correction documentation
- sklearn documentation for machine learning

### Next Steps:
1. Try this pipeline on real proteomics data (e.g., from PRIDE database)
2. Implement more sophisticated imputation methods
3. Add pathway enrichment analysis
4. Build a web app to share results

---

### Questions or Issues?

If you encounter problems:
1. Check that all cells ran in order (Runtime → Run all)
2. Verify random seed is set (ensures reproducibility)
3. Try restarting runtime if issues persist

Happy analyzing! 🧬📊

In [ ]:
## End of Notebook ##